In [21]:
pip install tensorflow keras cv2 matplotlib

  Using cached tensorflow-2.16.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (4.1 kB)
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Load Libraries

In [22]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

## Facial Recognition Component

In [23]:
def load_facenet_model(model_path):
    """ Load the pre-trained FaceNet model. """
    try:
        model = load_model(model_path)
        print("FaceNet model loaded successfully.")
        return model
    except Exception as e:
        print(f"Failed to load model: {e}")
        return None

# Using Fawkes to Poison Images

In [24]:
def poison_image_with_fawkes(input_image_path, output_dir):
    """ Use Fawkes to 'poison' the image to protect privacy. """
    output_image_path = os.path.join(output_dir, os.path.basename(input_image_path))
    command = f'fawkes -i {input_image_path} -o {output_dir} --mode minimal'
    subprocess.run(command, shell=True)
    return output_image_path

# Load and Prepare Images

In [25]:
def load_and_prepare_image(image_path, target_size=(160, 160)):
    """ Load and prepare an image file for FaceNet. """
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    return img_array_expanded / 255.0

# Using the FairFace Dataset

In [26]:
import cv2
import numpy as np

def load_and_prepare_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (160, 160))
    return image.astype('float32') / 255.0

# Testing Fairness and Effectiveness

In [31]:
def test_effectiveness(model, dataset_path, output_dir):
    """ Test the effectiveness of the 'poisoned' images against FaceNet model. """
    results = {}
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for image_file in os.listdir(dataset_path):
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            original_image_path = os.path.join(dataset_path, image_file)
            poisoned_image_path = poison_image_with_fawkes(original_image_path, output_dir)

            original_image = load_and_prepare_image(original_image_path)
            poisoned_image = load_and_prepare_image(poisoned_image_path)

            original_pred = model.predict(original_image)
            poisoned_pred = model.predict(poisoned_image)

            distance = np.linalg.norm(original_pred - poisoned_pred)
            results[image_file] = distance
            print(f"Processed {image_file}: Distance = {distance}")

    return results

# Execution

In [32]:
if __name__ == "__main__":
    facenet_model_path = './facenet_keras.h5'
    fairface_dataset_path = 'fairface'
    output_directory = 'output'

    # Load the FaceNet model
    model = load_facenet_model(facenet_model_path)

    if model is not None:
        # Run the effectiveness test
        effectiveness_results = test_effectiveness(model, fairface_dataset_path, output_directory)
        print(effectiveness_results)

Failed to load model: bad marshal data (unknown type code)
